### Using SQL and Bash
To store the csv file into a database

In [1]:
! apt-get update
! apt-get install mysql-server

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pac

In [2]:
!mysql --version # Making sure everything has been installed correctly by checking the version
!service mysql start # Starting our server

mysql  Ver 8.0.41-0ubuntu0.22.04.1 for Linux on x86_64 ((Ubuntu))
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [14]:
!mysql -e "SET GLOBAL local_infile = 1;"

In [5]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Copy file to correct folder so sql can access
# Change directory to where you stored filtered_flights.csv
!cp '/content/drive/MyDrive/Colab Notebooks/filtered_flights.csv' /var/lib/mysql-files/

In [12]:
! wc -l /var/lib/mysql-files/filtered_flights.csv

17440799 /var/lib/mysql-files/filtered_flights.csv


In [16]:
# Downloaded file locally to be able to use in local environments
files.download('/content/drive/MyDrive/Colab Notebooks/filtered_flights.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Used these commands to setup database in SQL shell and run queries:
show databases;<br>
CREATE DATABASE Flights;<br>
use Flights;<br>

**Creates the flights table**<br>
CREATE TABLE flights (
    FlightDate DATE,
    Airline VARCHAR(100),
    Origin VARCHAR(5),
    Dest VARCHAR(5),
    Cancelled INT,
    Diverted INT,
    DepTime INT,
    DepDelay INT,
    ArrTime INT,
    AirTime INT,
    Distance INT,
    Year INT,
    Quarter INT,
    Month INT,
    OriginCityName VARCHAR(100),
    OriginStateName VARCHAR(100),
    DestCityName VARCHAR(100),
    DestStateName VARCHAR(100),
    TaxiOut INT,
    WheelsOff INT,
    WheelsOn INT,
    TaxiIn INT,
    ArrDelay INT,
    DivAirportLandings INT
);<br>

**Load file into database**

LOAD DATA INFILE '/var/lib/mysql-files/filtered_flights.csv' INTO TABLE flights FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 ROWS;
<br>
**Queries that were ran:**<br>
Total Departures per Month per Year:<br>
SELECT Year, Month, COUNT(*) AS TotalDepartures
FROM flights
GROUP BY Year, Month
ORDER BY Year, TotalDepartures DESC;<br><br>

Average Delay per Airline over all years:<br>
SELECT Airline, AVG(ArrDelay+DepDelay) AS AvgDelay
FROM flights
GROUP BY Airline
ORDER BY AvgDelay DESC;<br><br>

Total Distance travelled per Airline:<br>
SELECT Airline, SUM(Distance) AS TotalDistance
FROM flights
GROUP BY Airline
ORDER BY TotalDistance DESC
LIMIT 10;<br><br>

Busiest Cities:<br>
SELECT OriginCityName, COUNT(*) AS TotalDepartures
FROM flights
GROUP BY OriginCityName
ORDER BY TotalDepartures DESC;<br><br>

Total Departures per Year:<br>
SELECT Year, COUNT(*) AS TotalDepartures
FROM flights
GROUP BY Year;

In [15]:
!mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 14
Server version: 8.0.41-0ubuntu0.22.04.1 (Ubuntu)

Copyright (c) 2000, 2025, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> use Flights;
Reading table information for completion of table and column names
You can turn off this feature to get a quicker startup with -A

Database changed
mysql> LOAD DATA LOCAL INFILE '/var/lib/mysql-files/filtered_flights.csv' INTO TABLE flights FIELDS  TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 ROWS;
ERROR 2068 (HY000): LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.
mysql> LOAD DATA INFILE '/var/lib/mysql-files/filtered_flights.csv' INTO TABLE flights FIELDS TERMIN ATED BY ',' ENCLOSED BY '"' LINES TERMINATED